In [29]:
!pip install pandera

In [30]:
import pandas as pd
import numpy as np
import pandera as pa

In [31]:
df = pd.read_csv('https://storage.googleapis.com/mkt-campaign/marketing_campaign.csv%20-%20marketing_campaign.csv.csv',
                encoding='utf8',
                index_col='ID',
                parse_dates=['Dt_Customer'],
                dayfirst=True)

df.sort_index(inplace=True)
df

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
ID,,,,,,,,,,,,,,,,,,,,,
0,1985,Graduation,Married,70951.0,0,0,2013-05-04,66,239,10,...,1,0,0,0,0,0,0,3,11,0
1,1961,Graduation,Single,57091.0,0,0,2014-06-15,0,464,5,...,5,0,0,0,0,1,0,3,11,1
9,1975,Master,Single,46098.0,1,1,2012-08-18,86,57,0,...,8,0,0,0,0,0,0,3,11,0
13,1947,PhD,Widow,25358.0,0,1,2013-07-22,57,19,0,...,6,0,0,0,0,0,0,3,11,0
17,1971,PhD,Married,60491.0,0,1,2013-09-06,81,637,47,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11178,1972,Master,Single,42394.0,1,0,2014-03-23,69,15,2,...,7,0,0,0,0,0,0,3,11,0
11181,1949,PhD,Married,156924.0,0,0,2013-08-29,85,2,1,...,0,0,0,0,0,0,0,3,11,0
11187,1978,Basic,Single,26487.0,1,0,2013-05-20,23,2,8,...,5,0,0,0,0,0,0,3,11,0


In [32]:
df.rename(columns={
'Year_Birth':'Ano_Nascimento',
'Education':'Educacao',
'Marital_Status':'Estado_Civil',
'Income':'Renda_Anual',
'Kidhome':'Qtd_Crianca_Pequena',
'Teenhome':'Qtd_Adolescente',
'Dt_Customer':'Data_Cliente',
'Recency':'Dias_Ultima_Compra',
'MntWines':'GastoProdVinhos',
'MntFruits':'GastoProdFrutas',
'MntMeatProducts':'GastoProdCarnes',
'MntFishProducts':'GastoProdPeixes',
'MntSweetProducts':'GastoProdDoces',
'MntGoldProds':'GastoProdOuro',
'NumDealsPurchases':'QtdComprasDesconto',
'NumWebPurchases':'QtdComprasInternet',
'NumCatalogPurchases':'QtdComprasCatalogo',
'NumStorePurchases':'QtdComprasLoja',
'NumWebVisitsMonth':'QtdVititasSiteMes',
'AcceptedCmp3':'AdesaoCamp3',
'AcceptedCmp4':'AdesaoCamp4',
'AcceptedCmp5':'AdesaoCamp5',
'AcceptedCmp1':'AdesaoCamp1',
'AcceptedCmp2':'AdesaoCamp2',
'Complain':'Reclamacao',
'Z_CostContact':'Z_CustoContato',
'Z_Revenue':'Z_Receita',
'Response':'Resposta'
}, inplace=True)

In [33]:
#Verificando os tipos das colunas

df.info()

#Após a verificação, apenas as colunas "Educacao, Estado_Civil" serão traduzidas. As outras colunas são de tipo númerico/data.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 0 to 11191
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Ano_Nascimento       2240 non-null   int64         
 1   Educacao             2240 non-null   object        
 2   Estado_Civil         2240 non-null   object        
 3   Renda_Anual          2216 non-null   float64       
 4   Qtd_Crianca_Pequena  2240 non-null   int64         
 5   Qtd_Adolescente      2240 non-null   int64         
 6   Data_Cliente         2240 non-null   datetime64[ns]
 7   Dias_Ultima_Compra   2240 non-null   int64         
 8   GastoProdVinhos      2240 non-null   int64         
 9   GastoProdFrutas      2240 non-null   int64         
 10  GastoProdCarnes      2240 non-null   int64         
 11  GastoProdPeixes      2240 non-null   int64         
 12  GastoProdDoces       2240 non-null   int64         
 13  GastoProdOuro        2240 non-nu

In [34]:
#Separação das colunas com o tipo "object" para tratamento de tradução

colTraducao = df.select_dtypes(include=['object'])

#Criando variáveis com valores originais e traduzidos

#Tratando coluna Educacao
educacao = colTraducao['Educacao'].unique() #Unique é uma função que separa os valores destintos da coluna
traducaoEdu = ['Superior Completo', 'Mestre', 'Doutor', 'Segundo Ciclo', 'Ensino Fundamental'] #Criação de uma lista com os valores traduzidos

#Tratando coluna Estado_Civil

estadoCivil = colTraducao['Estado_Civil'].unique()
traducaoEC = ['Casado', 'Solteiro', 'Viúvo', 'Divorciado', 'Uniao estável', 'Sozinho', 'YOLO', 'Absurdo']

#Chamando a função replace para tradução das linhas

df.replace(educacao, traducaoEdu, inplace=True)
df.replace(estadoCivil, traducaoEC, inplace=True)




In [35]:
#Apenas a coluna Renda_Anual possui valores nulos
df.isnull().sum()

Ano_Nascimento          0
Educacao                0
Estado_Civil            0
Renda_Anual            24
Qtd_Crianca_Pequena     0
Qtd_Adolescente         0
Data_Cliente            0
Dias_Ultima_Compra      0
GastoProdVinhos         0
GastoProdFrutas         0
GastoProdCarnes         0
GastoProdPeixes         0
GastoProdDoces          0
GastoProdOuro           0
QtdComprasDesconto      0
QtdComprasInternet      0
QtdComprasCatalogo      0
QtdComprasLoja          0
QtdVititasSiteMes       0
AdesaoCamp3             0
AdesaoCamp4             0
AdesaoCamp5             0
AdesaoCamp1             0
AdesaoCamp2             0
Reclamacao              0
Z_CustoContato          0
Z_Receita               0
Resposta                0
dtype: int64

In [36]:
#Valores nulos excluidos por não representar valor à análise
df.dropna(axis=0, how='any', inplace=True)

In [37]:
df.drop(['Z_CustoContato', 'Z_Receita', 'Resposta'], axis='columns' , inplace=True)

In [38]:
schema = {
'Ano_Nascimento':pa.Column(pa.Int),
'Educacao':pa.Column(pa.String), 
'Estado_Civil':pa.Column(pa.String), 
'Renda_Anual':pa.Column(pa.Float),
'Qtd_Crianca_Pequena':pa.Column(pa.Int), 
'Qtd_Adolescente':pa.Column(pa.Int), 
'Data_Cliente':pa.Column(pa.DateTime),
'Dias_Ultima_Compra':pa.Column(pa.Int), 
'GastoProdVinhos':pa.Column(pa.Int), 
'GastoProdFrutas':pa.Column(pa.Int),
'GastoProdCarnes':pa.Column(pa.Int), 
'GastoProdPeixes':pa.Column(pa.Int), 
'GastoProdDoces':pa.Column(pa.Int), 
'GastoProdOuro':pa.Column(pa.Int),
'QtdComprasDesconto':pa.Column(pa.Int), 
'QtdComprasInternet':pa.Column(pa.Int), 
'QtdComprasCatalogo':pa.Column(pa.Int),
'QtdComprasLoja':pa.Column(pa.Int), 
'QtdVititasSiteMes':pa.Column(pa.Int), 
'AdesaoCamp3':pa.Column(pa.Int), 
'AdesaoCamp4':pa.Column(pa.Int),
'AdesaoCamp5':pa.Column(pa.Int), 
'AdesaoCamp1':pa.Column(pa.Int), 
'AdesaoCamp2':pa.Column(pa.Int), 
'Reclamacao':pa.Column(pa.Int)
}

schemaDF = pa.DataFrameSchema(columns=schema)

schemaDF.validate(df)

,Ano_Nascimento,Educacao,Estado_Civil,Renda_Anual,Qtd_Crianca_Pequena,Qtd_Adolescente,Data_Cliente,Dias_Ultima_Compra,GastoProdVinhos,GastoProdFrutas,...,QtdComprasInternet,QtdComprasCatalogo,QtdComprasLoja,QtdVititasSiteMes,AdesaoCamp3,AdesaoCamp4,AdesaoCamp5,AdesaoCamp1,AdesaoCamp2,Reclamacao
ID,,,,,,,,,,,,,,,,,,,,,
0,1985,Superior Completo,Casado,70951.0,0,0,2013-05-04,66,239,10,...,3,4,9,1,0,0,0,0,0,0
1,1961,Superior Completo,Solteiro,57091.0,0,0,2014-06-15,0,464,5,...,7,3,7,5,0,0,0,0,1,0
9,1975,Mestre,Solteiro,46098.0,1,1,2012-08-18,86,57,0,...,3,2,2,8,0,0,0,0,0,0
13,1947,Doutor,Viúvo,25358.0,0,1,2013-07-22,57,19,0,...,1,0,3,6,0,0,0,0,0,0
17,1971,Doutor,Casado,60491.0,0,1,2013-09-06,81,637,47,...,6,11,7,5,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11178,1972,Mestre,Solteiro,42394.0,1,0,2014-03-23,69,15,2,...,1,0,3,7,0,0,0,0,0,0
11181,1949,Doutor,Casado,156924.0,0,0,2013-08-29,85,2,1,...,0,0,0,0,0,0,0,0,0,0
11187,1978,Ensino Fundamental,Solteiro,26487.0,1,0,2013-05-20,23,2,8,...,2,1,3,5,0,0,0,0,0,0


In [39]:
display(df)

,Ano_Nascimento,Educacao,Estado_Civil,Renda_Anual,Qtd_Crianca_Pequena,Qtd_Adolescente,Data_Cliente,Dias_Ultima_Compra,GastoProdVinhos,GastoProdFrutas,...,QtdComprasInternet,QtdComprasCatalogo,QtdComprasLoja,QtdVititasSiteMes,AdesaoCamp3,AdesaoCamp4,AdesaoCamp5,AdesaoCamp1,AdesaoCamp2,Reclamacao
ID,,,,,,,,,,,,,,,,,,,,,
0,1985,Superior Completo,Casado,70951.0,0,0,2013-05-04,66,239,10,...,3,4,9,1,0,0,0,0,0,0
1,1961,Superior Completo,Solteiro,57091.0,0,0,2014-06-15,0,464,5,...,7,3,7,5,0,0,0,0,1,0
9,1975,Mestre,Solteiro,46098.0,1,1,2012-08-18,86,57,0,...,3,2,2,8,0,0,0,0,0,0
13,1947,Doutor,Viúvo,25358.0,0,1,2013-07-22,57,19,0,...,1,0,3,6,0,0,0,0,0,0
17,1971,Doutor,Casado,60491.0,0,1,2013-09-06,81,637,47,...,6,11,7,5,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11178,1972,Mestre,Solteiro,42394.0,1,0,2014-03-23,69,15,2,...,1,0,3,7,0,0,0,0,0,0
11181,1949,Doutor,Casado,156924.0,0,0,2013-08-29,85,2,1,...,0,0,0,0,0,0,0,0,0,0
11187,1978,Ensino Fundamental,Solteiro,26487.0,1,0,2013-05-20,23,2,8,...,2,1,3,5,0,0,0,0,0,0


In [40]:
df.to_csv('gs://mkt-campaign/saida/mkt.csv')